In [3]:
using JuMP, Gurobi, DataFrames, CSV, Plots

# Read in Data and set Parameters

In [5]:
data1 = CSV.read("Scenario1.csv")
data2 = CSV.read("Scenario2.csv")
data3 = CSV.read("Scenario3.csv")
data4 = CSV.read("Scenario4.csv");

In [6]:
start_stations = Set(unique(data1[!,2]))
end_stations = Set(unique(data1[!,3]))
stations = collect(union(start_stations,end_stations))
time_intervals = unique(data1[!,8])
days = unique(data1[!,7]);

In [7]:
#define sizes 
# m = total number of start station clusters
m= size(stations,1)
# n = total number of time windows
n = size(time_intervals,1)
# o = total number of days
o = size(days,1);


In [62]:
r_ride = 1.15
c_bike = 200;

# Optimization Model

In [63]:
function optimize_distribution(data)
    #initialize model
    mod = Model(solver = GurobiSolver())

    #decision variables
    @variable(mod,x[i=1:m,t=1:n,d=1:o]>= 0)
    @variable(mod,d[i=1:m,j=1:m,t=1:n,l=1:o] >= 0)

    #objective function
    @objective(mod,Max,sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o))


    #constraints
    @constraint(mod,[i=1:m,t=1:n-1,l=1:o],x[i,t,l] - sum(d[i,j,t,l] for j=1:m) + sum(d[j,i,t,l] for j=1:m)-x[i,t+1,l]==0)
    @constraint(mod,[d=1:o], sum(x[i,1,d] for i=1:m) == sum(x[i,1,1] for i=1:m))
    @constraint(mod,[i=1:size(data,1)],d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1,data[i,:Day]] <= data[i,:dep_flow])
    @constraint(mod,[i=1:m,t=1:n,l=1:o],d[i,i,t,l]==0)
    @constraint(mod,r_ride*sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o) - c_bike*sum(x[i,1,1] for i=1:m) >= -200000)



    #solve model
    status = solve(mod)
    
    return sum(getvalue(x)[:,1,1]),getvalue(d)
end



optimize_distribution (generic function with 1 method)

In [64]:
function compute_missed_rides(data,d)
    missed_rides = [abs(data[i,:dep_flow]-d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1,data[i,:Day]])
    for i=1:size(data,1)]
    return sum(missed_rides)
end

compute_missed_rides (generic function with 1 method)

In [65]:
opt_scen1 = optimize_distribution(data1)
opt_scen2 = optimize_distribution(data2)
opt_scen3 = optimize_distribution(data3)
opt_scen4 = optimize_distribution(data4)

Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 1870531 rows, 1836000 columns and 7088950 nonzeros
Model fingerprint: 0x3ed6bcd5
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+05]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 1852952 rows and 1747304 columns
Presolve time: 5.34s
Presolved: 17579 rows, 88696 columns, 249173 nonzeros

Ordering time: 0.03s

Barrier statistics:
 AA' NZ     : 9.836e+04
 Factor NZ  : 7.691e+05 (roughly 50 MBytes of memory)
 Factor Ops : 4.827e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   5.19822596e+07  4.5105

(419.0, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

**Compute total number of allowed rides**

In [66]:
println(sum(opt_scen1[2]))
println(sum(opt_scen2[2]))
println(sum(opt_scen3[2]))
println(sum(opt_scen4[2]))

100509.0
58271.0
125627.0
61232.0


#### Compute Minimum Bikes for Each Scenario

In [67]:
min_bikes1 = opt_scen1[1]
println(min_bikes1)
min_bikes2 = opt_scen2[1]
println(min_bikes2)
min_bikes3 = opt_scen3[1]
println(min_bikes3)
min_bikes4 = opt_scen4[1]
println(min_bikes4)

564.0
441.0
629.0
419.0


#### Compute Missed Rides for Each Scenario

In [68]:
missed_rides1 = compute_missed_rides(data1,opt_scen1[2])
println(missed_rides1)
missed_rides2 = compute_missed_rides(data2,opt_scen2[2])
println(missed_rides2)
missed_rides3 = compute_missed_rides(data3,opt_scen3[2])
println(missed_rides3)
missed_rides4 = compute_missed_rides(data4,opt_scen4[2])
println(missed_rides4)

9845.0
6556.0
11615.0
6529.0


# Baseline Model

In [69]:
function baseline_distribution(min_bikes,data)
    uniform_dist = round(min_bikes/m)
    mod = Model(solver = GurobiSolver())

    #decision variables
    @variable(mod,x[i=1:m,t=1:n,l=1:o]>= 0)
    @variable(mod,d[i=1:m,j=1:m,t=1:n,l=1:o] >= 0)

    #objective function
    @objective(mod,Max,sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o))


    #constraints
    @constraint(mod,[i=1:m,t=1:n-1,l=1:o],x[i,t,l] - sum(d[i,j,t,l] for j=1:m) + sum(d[j,i,t,l] for j=1:m)-x[i,t+1,l]==0)
    @constraint(mod,[d=1:o], sum(x[i,1,d] for i=1:m) == sum(x[i,1,1] for i=1:m))
    @constraint(mod,[i=1:size(data,1)],d[data[i,:start_area],data[i,:end_area],data[i,:Hour]+1,data[i,:Day]] <= data[i,:dep_flow])
    @constraint(mod,[i=1:m,t=1:n,l=1:o],d[i,i,t,l]==0)
    @constraint(mod,[i=1:m,d=1:o],x[i,1,d]==uniform_dist)
    @constraint(mod,r_ride*sum(d[i,j,t,l] for i=1:m,j=1:m,t=1:n,l=1:o) - c_bike*sum(x[i,1,1] for i=1:m) >= -200000)

    status = solve(mod)
    
    return getvalue(d)
end


baseline_distribution (generic function with 1 method)

In [70]:
baseline_scen1 = baseline_distribution(min_bikes1,data1)
baseline_scen2 = baseline_distribution(min_bikes2,data2)
baseline_scen3 = baseline_distribution(min_bikes3,data3)
baseline_scen4 = baseline_distribution(min_bikes4,data4);

Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 1872031 rows, 1836000 columns and 7090450 nonzeros
Model fingerprint: 0x1b67f93a
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+05]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 1858760 rows and 1755414 columns
Presolve time: 5.06s
Presolved: 13271 rows, 80586 columns, 159630 nonzeros

Ordering time: 0.02s

Barrier statistics:
 AA' NZ     : 7.005e+04
 Factor NZ  : 5.286e+05 (roughly 40 MBytes of memory)
 Factor Ops : 2.806e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.55814804e+06  2.4357

#### Compute Missed Rides for Each Scenario

In [71]:
baseline_missed1 = compute_missed_rides(data1,baseline_scen1)
println(baseline_missed1)
baseline_missed2 = compute_missed_rides(data2,baseline_scen2)
println(baseline_missed2)
baseline_missed3 = compute_missed_rides(data3,baseline_scen3)
println(baseline_missed3)
baseline_missed4 = compute_missed_rides(data4,baseline_scen4)
println(baseline_missed4)

13816.0
8612.0
16262.0
9377.0


#### Compute Revenue Gains through Optimal Distribution for Each Scenario

In [73]:
ride_cost = 3.25
revenue1 = ride_cost*(baseline_missed1-missed_rides1)
println(revenue1)
revenue2 = ride_cost*(baseline_missed2-missed_rides2)
println(revenue2)
revenue3 = ride_cost*(baseline_missed3-missed_rides3)
println(revenue3)
revenue4 = ride_cost*(baseline_missed4-missed_rides4)
println(revenue4)


12905.75
6682.0
15102.75
9256.0


In [72]:
println(sum(baseline_scen1))
println(sum(baseline_scen2))
println(sum(baseline_scen3))
println(sum(baseline_scen4))

96538.0
56215.0
120980.0
58384.0
